In [4]:
!pip install numpy pandas matplotlib torch torchmetrics

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 960.0 kB/s eta 0:00:14
     ---------------------------------------- 0.1/12.9 MB 1.1 MB/s eta 0:00:12
      --------------------------------------- 0.2/12.9 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.4/12.9 MB 2.1 MB/s eta 0:00:07
     - -------------------------------------- 0.5/12.9 MB 2.4 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/12.9 MB 2.8 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/12.9 MB 2.9 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.9 MB 2.8 MB/s eta 0:00:05
     --- ------------------------------------ 1.3/12.9 MB 3.1 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/12.9 MB 3.2 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/12.9 MB 3.2 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/12.9 MB 

In [5]:
import pandas as pd
from pathlib import Path

# Path to the small transactions CSV (relative to this notebook).
DATA_PATH = Path("dataset") / "HI-Small_Trans.csv"

# Load into a DataFrame
small_trans = pd.read_csv(DATA_PATH)

# Quick summary and preview
print(f"Loaded {len(small_trans)} rows; columns: {list(small_trans.columns)}")
small_trans.head()

Loaded 5078345 rows; columns: ['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1', 'Amount Received', 'Receiving Currency', 'Amount Paid', 'Payment Currency', 'Payment Format', 'Is Laundering']


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [10]:
# Basic analysis: currencies, banks, and other summaries
# This cell is robust to different column names: it searches for currency-like and bank-like columns

# Show shape and a small sample
rows, cols = small_trans.shape
print(f"Data shape: {rows} rows x {cols} columns")
print()
print("Sample rows:")
display(small_trans.head())

# Missing values by column (top 10)
missing_by_col = small_trans.isnull().sum().sort_values(ascending=False).head(15)
print("Top missing values by column:")
print(missing_by_col.to_string())
print()

# Find likely currency column(s)
currency_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('currency','ccy','curr'))]
if currency_candidates:
    cur_col = currency_candidates[0]
    num_currencies = small_trans[cur_col].nunique(dropna=True)
    top_currencies = small_trans[cur_col].value_counts().head(10)
    print(f"Found currency column: '{cur_col}' — {num_currencies} unique values")
    print("Top currencies (by count):")
    print(top_currencies.to_string())
else:
    cur_col = None
    print("No currency-like column found.")
    print("Columns:", list(small_trans.columns))

print()
# Find likely bank-related columns
bank_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('bank','institution','bic','iban','bankid','bank_id','bankname','bank_name'))]
if bank_candidates:
    # Count unique bank identifiers across candidate columns
    unique_banks = set()
    for c in bank_candidates:
        unique_banks.update(small_trans[c].dropna().astype(str).unique())
    num_unique_banks = len(unique_banks)
    print(f"Found bank-like columns: {bank_candidates} — approx. {num_unique_banks} unique bank identifiers (aggregated)")
else:
    num_unique_banks = None
    print("No bank-like columns found.")

print()
# Other basic summaries: amount column candidates and top senders/receivers if available
amt_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('amount','amt','value'))]
if amt_candidates:
    amt_col = amt_candidates[0]
    print(f"Found amount column: {amt_col} — summary:")
    print(small_trans[amt_col].describe())
else:
    print("No amount-like column found.")

# If sender/receiver columns exist, show top participants
party_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('sender','receiver','originator','beneficiary','from_','to_','account'))]
if party_candidates:
    print()
    print("Top participants in party-like columns:")
    for c in party_candidates:
        print(f"Column: {c}")
# Final small sample
display(small_trans.sample(min(5, len(small_trans))))

Data shape: 5078345 rows x 11 columns

Sample rows:


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


Top missing values by column:
Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0

Found currency column: 'Receiving Currency' — 15 unique values
Top currencies (by count):
Receiving Currency
US Dollar      1879341
Euro           1172017
Swiss Franc     237884
Yuan            206551
Shekel          194988
Rupee           192065
UK Pound        181255
Ruble           157361
Yen             156319
Bitcoin         148151

Found bank-like columns: ['From Bank', 'To Bank'] — approx. 30470 unique bank identifiers (aggregated)

Found amount column: Amount Received — summary:
count    5.078345e+06
mean     5.988726e+06
std      1.037183e+09
min      1.000000e-06
25%      1.833700e+02
50%      1.411010e+03
75%      1.234627e+04
max      1.046302e+12
Name: Amount Received, dtype: float64

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
804548,2022/09/01 14:39,117698,807FDDD80,117698,807FDDD80,127679.82,Euro,127679.82,Euro,Reinvestment,0
1791198,2022/09/02 21:04,27,80C17CFE0,33739,80CC21F20,13746.26,Mexican Peso,13746.26,Mexican Peso,Credit Card,0
1791630,2022/09/02 21:25,38478,80E88C760,239453,80E8EF9D0,1438702.67,Swiss Franc,1438702.67,Swiss Franc,Cash,0
4886363,2022/09/10 00:03,240304,80EA8D930,139298,80E9C4CC0,26.36,Swiss Franc,26.36,Swiss Franc,Credit Card,0
2449987,2022/09/05 07:49,2843,80068D8B0,238829,813476AF0,935.63,US Dollar,935.63,US Dollar,Credit Card,0


In [1]:
# Minimal PyTorch CUDA availability check - fail fast if CUDA is unavailable
try:
    import torch
except Exception as e:
    raise RuntimeError(f"PyTorch is not installed or failed to import: {e}")

# Ensure CUDA is available - if not, raise an error because we rely on GPU
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please ensure an NVIDIA GPU, drivers, and CUDA are installed and that the CUDA toolkit is compatible with your PyTorch build.")

# Report devices
print('PyTorch version:', torch.__version__)
cnt = torch.cuda.device_count()
print('CUDA device count:', cnt)
for i in range(cnt):
    try:
        name = torch.cuda.get_device_name(i)
    except Exception:
        name = f'unknown-device-{i}'
    print(f'Device {i}:', name)

# Quick allocation test to confirm functional GPU access
try:
    _ = torch.zeros(1, device='cuda')
    print('Successfully allocated a tensor on CUDA.')
except Exception as e:
    raise RuntimeError('CUDA appears available but tensor allocation failed: ' + str(e))


RuntimeError: CUDA is not available. Please ensure an NVIDIA GPU, drivers, and CUDA are installed and that the CUDA toolkit is compatible with your PyTorch build.

In [3]:
"""cuda_diagnose.py
Small diagnostic script to help identify why CUDA (PyTorch) is not available on a machine

How to run (PowerShell):
  python .\cuda_diagnose.py

This script checks:
 - `nvidia-smi` output (driver and CUDA reported by the driver)
 - `nvcc --version` (if nvcc is in PATH)
 - common CUDA environment variables (CUDA_PATH, CUDA_HOME) and PATH entries
 - PyTorch import, torch.__version__, torch.version.cuda, torch.cuda.is_available(), and device names

It prints clear, actionable messages. It does not modify the system.
"""
from __future__ import annotations

import os
import platform
import shutil
import subprocess
import sys
from typing import Optional, Tuple


def run_cmd(cmd: list[str]) -> Tuple[int, str]:
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=False)
        return 0, out.decode(errors="ignore").strip()
    except subprocess.CalledProcessError as e:
        return e.returncode, (e.output.decode(errors="ignore") if e.output is not None else str(e))
    except FileNotFoundError:
        return 127, "command not found"
    except Exception as e:
        return 1, str(e)


def print_header(title: str) -> None:
    print("\n" + "=" * 8 + f" {title} " + "=" * 8)


def check_nvidia_smi() -> Optional[str]:
    nvsmi = shutil.which("nvidia-smi")
    if not nvsmi:
        print("nvidia-smi not found in PATH.")
        return None

    print(f"nvidia-smi found at: {nvsmi}")
    rc, out = run_cmd([nvsmi, "-L"])  # list GPUs
    if rc == 0 and out:
        print("nvidia-smi -L output:")
        print(out)
    else:
        # try a more verbose query to capture driver and CUDA version
        rc2, out2 = run_cmd([nvsmi, "--query-gpu=name,driver_version,cuda_version", "--format=csv,noheader"])
        if rc2 == 0 and out2:
            print("nvidia-smi query output:")
            print(out2)
            return out2
        else:
            # fallback: print full nvidia-smi
            rc3, out3 = run_cmd([nvsmi])
            if rc3 == 0:
                print(out3.splitlines()[:20])
            else:
                print("Failed to run nvidia-smi: ", out3)
    return out


def check_nvcc() -> Optional[str]:
    nvcc = shutil.which("nvcc")
    if not nvcc:
        print("nvcc not found in PATH (CUDA toolkit may not be installed or not on PATH).")
        return None
    print(f"nvcc found at: {nvcc}")
    rc, out = run_cmd([nvcc, "--version"])
    if rc == 0:
        print("nvcc --version:\n", out)
        return out
    else:
        print("nvcc returned non-zero (unable to get version):", out)
        return None


def check_env_vars() -> None:
    print("\nChecking CUDA-related environment variables and PATH entries:")
    vars_of_interest = ["CUDA_PATH", "CUDA_HOME", "CUDA_ROOT", "PATH"]
    for v in vars_of_interest:
        val = os.environ.get(v)
        if val:
            if v == "PATH":
                # only show PATH entries that look relevant
                parts = val.split(os.pathsep)
                cuda_parts = [p for p in parts if "cuda" in p.lower() or "nvidia" in p.lower()]
                print(f"{v} (matching): {cuda_parts}")
            else:
                print(f"{v}: {val}")
        else:
            print(f"{v}: (not set)")


def check_pytorch() -> bool:
    try:
        import torch
    except Exception as e:
        print('\nPyTorch import failed:', e)
        print('If you expect PyTorch to be available, ensure the correct python environment is active and torch is installed (pip show torch).')
        return False

    print_header("PyTorch")
    print("torch.__version__:", getattr(torch, "__version__", "<unknown>"))
    print("torch.version.cuda:", getattr(torch.version, "cuda", None))
    try:
        cuda_avail = torch.cuda.is_available()
    except Exception as e:
        cuda_avail = False
        print('torch.cuda.is_available() raised:', e)
    print("torch.cuda.is_available():", cuda_avail)

    if cuda_avail:
        try:
            cnt = torch.cuda.device_count()
            print("torch.cuda.device_count():", cnt)
            for i in range(cnt):
                try:
                    print(f"  device {i}:", torch.cuda.get_device_name(i))
                except Exception:
                    print(f"  device {i}: <name unknown>")
        except Exception as e:
            print("Error querying devices:", e)

        # cummulative: check cudnn version if present
        try:
            cudnn_ver = torch.backends.cudnn.version()
            print("cudnn version:", cudnn_ver)
        except Exception:
            pass

        # try small allocation to validate functionality
        try:
            t = torch.zeros(1, device="cuda")
            del t
            print("Successfully allocated a small tensor on CUDA.")
        except Exception as e:
            print("Allocation on CUDA failed:", e)
            return False

        return True

    return False


def main() -> int:
    print_header("Environment")
    print("Platform:", platform.platform())
    print("Python:", sys.version.replace('\n', ' '))

    print_header("nvidia-smi")
    nvsmi_info = check_nvidia_smi()

    print_header("nvcc")
    nvcc_info = check_nvcc()

    check_env_vars()

    # PyTorch check
    ok = check_pytorch()

    print_header("Summary & next steps")
    if ok:
        print("CUDA looks available to PyTorch. If you still get errors, verify your script is running under the same Python environment where torch was checked.")
        return 0

    # not ok
    print("\nPyTorch cannot use CUDA. Common causes and tips:")
    print(" - PyTorch binary was installed for a different CUDA version than the NVIDIA driver supports.")
    print(" - NVIDIA driver is missing or too old (check nvidia-smi driver version).")
    print(" - CUDA toolkit not in PATH (nvcc missing) or PATH points to conflicting CUDA installs.")
    print(" - Running inside an environment that doesn't have torch built with CUDA support (check 'pip show torch' and torch.version.cuda).")
    print("\nRecommended actions:")
    print(" 1) Run this script and paste key outputs (nvidia-smi, nvcc, torch.version.cuda).")
    print(" 2) Ensure NVIDIA driver is installed and up-to-date for your GPU (5080 requires recent drivers).")
    print(" 3) Install a PyTorch build that matches your CUDA environment; see https://pytorch.org/get-started/locally/")
    print(" 4) If using conda, prefer conda-installed cudatoolkit/torch combos or use the correct pip wheel for your CUDA version.")
    return 2


if __name__ == "__main__":
    rc = main()
    sys.exit(rc)


======== Environment ========
Platform: Windows-10-10.0.26200-SP0
Python: 3.10.10 | packaged by Anaconda, Inc. | (main, Mar 21 2023, 18:39:17) [MSC v.1916 64 bit (AMD64)]

======== nvidia-smi ========
nvidia-smi found at: C:\WINDOWS\system32\nvidia-smi.EXE
nvidia-smi -L output:
GPU 0: NVIDIA GeForce RTX 5080 (UUID: GPU-fca79032-2aa2-5888-3a54-32f2be4bbe8d)

======== nvcc ========
nvcc not found in PATH (CUDA toolkit may not be installed or not on PATH).

Checking CUDA-related environment variables and PATH entries:
CUDA_PATH: (not set)
CUDA_HOME: (not set)
CUDA_ROOT: (not set)
PATH (matching): ['C:\\Program Files\\NVIDIA Corporation\\NVIDIA app\\NvDLISR', 'C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common']

======== PyTorch ========
torch.__version__: 2.9.0+cpu
torch.version.cuda: None
torch.cuda.is_available(): False

======== Summary & next steps ========

PyTorch cannot use CUDA. Common causes and tips:
 - PyTorch binary was installed for a different CUDA version than the

SystemExit: 2

c:\Users\Max\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
